In [ ]:
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import obspy
import os
import glob

import hvsrpy
from hvsrpy import utils
%matplotlib inline

In [ ]:
### Time Domain Settings

windowlength = 60

filter_bool = False       
filter_flow = 0.1                   
filter_fhigh = 10                  
filter_order = 5

# Width of cosine taper {0. - 1.}. Geopsy default of 0.05 is equal to 0.1 -> 0.1 is recommended
# 余弦锥度宽度 {0. - 1.}。 Geopsy 默认值 0.05 等于 0.1 -> 建议使用 0.1
width = 0.1

### Frequency Domain Settings

# Konno and Ohmachi smoothing constant. 40 is recommended.
bandwidth = 40
# Minimum frequency after resampling
resample_fmin = 0.1
# Maximum frequency after resampling
resample_fmax = 10
# Number of frequencies after resampling 重采样后的频率点数
resample_fnum = 1000
# Type of resampling {'log', 'linear'}
resample_type = 'linear'
# Upper and lower frequency limits to restrict peak selection. To use the entire range use `None`.
peak_f_lower = None
peak_f_upper = None


### HVSR Settings

# Method for combining horizontal components {"squared-average", "geometric-mean", "single-azimuth"}.
# Geopsy's default is "squared-average" -> "geometric-mean" is recommended.
method = "geometric-mean"
# If method="single-azimuth", set azimuth in degree clock-wise from north. If method!="single-azimuth", value is ignored.
azimuth = 0

# Boolean to control whether frequency domain rejection proposed by Cox et al. (2020) is applied.
# Geopsy does not offer this functionality.
rejection_bool = True
# Number of standard deviations to consider during rejection. Smaller values will reject more windows -> 2 is recommended.
n = 2
# Maximum number of iterations to perform for rejection -> 50 is recommended.
max_iterations = 50

# Distribution of f0 {"lognormal", "normal"}. Geopsy default "normal" -> "lognormal" is recommended.
distribution_f0 = "lognormal"
# Distribution of mean curve {"lognormal", "normal"}. Geopsy default "lognormal" -> "lognormal" is recommended.
distribution_mc = "lognormal"


### Plot Settings

# Manually set the ylimits of the HVSR figures. Default is None so limits will be set automatically.
ymin, ymax = None, None

In [ ]:
def write_mseed(data_X, data_Y, data_Z, filename):
    Trace_X = obspy.Trace()
    Trace_Y = obspy.Trace()
    Trace_Z = obspy.Trace()

    Trace_Z.data = data_Z
    Trace_Y.data = data_Y
    Trace_X.data = data_X

    Trace_X.stats.starttime = obspy.UTCDateTime('2020-01-01T00:00:00.000000Z')
    Trace_Y.stats.starttime = obspy.UTCDateTime('2020-01-01T00:00:00.000000Z')
    Trace_Z.stats.starttime = obspy.UTCDateTime('2020-01-01T00:00:00.000000Z')

    Trace_X.stats.delta = 0.002
    Trace_Y.stats.delta = 0.002
    Trace_Z.stats.delta = 0.002

    st = obspy.Stream([Trace_X, Trace_Y, Trace_Z])
    st[0].meta.channel = 'BHZ'
    st[1].meta.channel = 'BHN'
    st[2].meta.channel = 'BHE'

    st.write(filename, format='MSEED')

In [ ]:
import pathlib
from scipy import io as sio

path_Z      = pathlib.Path('/media/user/Extreme Pro/YXMK/YXsac/z_h')
path_X      = pathlib.Path('/media/user/Extreme Pro/YXMK/YXsac/x_h')
path_Y      = pathlib.Path('/media/user/Extreme Pro/YXMK/YXsac/y_h')
path_mseed  = pathlib.Path('./YXMK/YXsac/YXmseed')

files_Z = path_Z.iterdir()
files_X = path_X.iterdir()
files_Y = path_Y.iterdir()



In [ ]:
for file_X, file_Y, file_Z in zip(sorted(files_X), sorted(files_Y), sorted(files_Z)):
    for X, Y, Z in zip(sorted(file_X.glob('*.*')), sorted(file_Y.glob('*.*')), sorted(file_Z.glob('*.*'))):

        data_X = np.array(list(np.squeeze(sio.loadmat(X)['hourdata'])), dtype=np.float64)
        data_Y = np.array(list(np.squeeze(sio.loadmat(Y)['hourdata'])), dtype=np.float64)
        data_Z = np.array(list(np.squeeze(sio.loadmat(Z)['hourdata'])), dtype=np.float64)
        
        dirname = path_mseed / file_X.name
        dirname.mkdir(parents=True, exist_ok=True)
        name = X.stem + '.mseed'
        filename = dirname / name
        print(filename)
        write_mseed(data_X, data_Y, data_Z, filename)